**Text Generation with LSTM** 

In [44]:
import numpy
# 1. What we want ot do is use the Sequence models as generative models 
# 2. What we have is sequence of words/charectors or a storybook in text format 
# 3. What we want to achieve is we want to generate a sentence or para from the storybook given few random 
#    charecotrs or words.
# We are going to consider the entire book as trian data.
# Because this is a non trevial problem with non repeateable sequences.
# Step:1 :- Load the text into memory
little_women = open("/Little_women.txt").read()
# For now I am just taking 1/8 th of the story 
little_women = little_women[10000:len(little_women)//8]

In [45]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
#from fastai import *
#master_bar, progress_bar = force_console_behavior()
#fastai.basic_train.master_bar, fastai.basic_train.progress_bar = master_bar, progress_bar

Found GPU at: /device:GPU:0


In [46]:
# Step 2:- geter the number of unique charectors 
# 2.1 Lower all the text from our storybook
little_women = little_women.lower()
# We have filterd only the alphabets from the 
chars =[char for char in list(set(little_women)) if(char.isalpha())] 
# Adding "." and "?" "," "space" and "'" in the vocabulary
# Also assigning a default char "dc"
default_char = "dc"
chars = chars+[" ",".","?","'",",",default_char]
# Getting the vocab length
vocab_len = len(chars)
total_chars = len(little_women)
print("Vocab length;- ",vocab_len)
print("Total number of charectors present:- ",total_chars)

Vocab length;-  32
Total number of charectors present:-  119059


In [47]:
# Step 3:- Assigning an integer number with each char 
char_to_int_mapping = {chars[i]:i for i in range(len(chars))}
# Step 4:- reversing the char to int to int to char
int_to_char_mapping = {integer:char for (char,integer) in char_to_int_mapping.items()}

In [48]:
# Step 5:- We are going to consider a sequence length of 100 
# also we create a sequence of integers with a corrosponding sequence of charectors
seq_len  = 100
X = []
y = []

for i in range(total_chars-seq_len):
  input_seq = little_women[i:seq_len+i]
  output_char = little_women[i+seq_len]
  X.append([char_to_int_mapping.get(char,char_to_int_mapping[default_char]) for char in input_seq])
  y.append(char_to_int_mapping.get(output_char,char_to_int_mapping[default_char]))
y = numpy.eye(vocab_len)[y]

In [49]:
# Step 6:- We can cunstruct the network now 
# Here we will use the combination of Convolution with LSTM to predict the next most likely charector
# and by repeating this cycle we will be able to complete the sentence/s

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Dense,MaxPool1D,LSTM,Embedding,Dropout,Flatten,TimeDistributed
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from tensorflow.keras.preprocessing import sequence

In [50]:
X = sequence.pad_sequences(X,seq_len)

In [51]:
# Defining callbacks
callbacks_ = [ModelCheckpoint("seq_to_seq.h5",monitor="loss",save_best_only=True),
              #ReduceLROnPlateau(monitor="loss",patience=5,factor=0.2,verbose=0),
              EarlyStopping(monitor="loss",patience=5,mode="auto",verbose=0)]

In [70]:
# cunstrecting the model 
# let's assume an embedding vector length of 100
embedding_vec_len = 100
lstm_model = Sequential()
lstm_model.add(Embedding(len(char_to_int_mapping)+1,embedding_vec_len,input_length=seq_len))
# We are using convolution because it will give us good performance and also help in extracting meaningful features
lstm_model.add(Conv1D(filters=32,kernel_size=4,activation='relu'))
# We can experiment with maxPooling , In my case it didn't work well.
#lstm_model.add(MaxPool1D(pool_size=2,padding="same"))
#lstm_model.add(TimeDistributed(Flatten()))
lstm_model.add(LSTM(250,activation="tanh",return_sequences=True))
#lstm_model.add(LSTM(250,activation="tanh",return_sequences=True))
lstm_model.add(LSTM(200,activation="tanh"))
lstm_model.add(Dense(len(char_to_int_mapping),activation="softmax"))
lstm_model.compile(loss="categorical_crossentropy",optimizer="adam")
lstm_model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 100, 100)          3300      
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 97, 32)            12832     
_________________________________________________________________
lstm_28 (LSTM)               (None, 97, 250)           283000    
_________________________________________________________________
lstm_29 (LSTM)               (None, 200)               360800    
_________________________________________________________________
dense_17 (Dense)             (None, 32)                6432      
Total params: 666,364
Trainable params: 666,364
Non-trainable params: 0
_________________________________________________________________


In [76]:
history = lstm_model.fit(X,y,batch_size=1280,epochs=100,callbacks=callbacks_,use_multiprocessing=True,workers=6)

Epoch 1/10
93/93 [==============================] - 46s 492ms/step - loss: 0.0727
Epoch 2/10
93/93 [==============================] - 45s 489ms/step - loss: 0.0376
Epoch 3/10
93/93 [==============================] - 45s 489ms/step - loss: 0.0246
Epoch 4/10
93/93 [==============================] - 45s 488ms/step - loss: 0.0194
Epoch 5/10
93/93 [==============================] - 45s 489ms/step - loss: 0.0168
Epoch 6/10
93/93 [==============================] - 45s 488ms/step - loss: 0.0150
Epoch 7/10
93/93 [==============================] - 45s 488ms/step - loss: 0.0137
Epoch 8/10
93/93 [==============================] - 45s 488ms/step - loss: 0.0126
Epoch 9/10
93/93 [==============================] - 45s 489ms/step - loss: 0.0116
Epoch 10/10
93/93 [==============================] - 45s 488ms/step - loss: 0.0108


In [83]:
# Step 8: Testing our text generation capabilities
start = numpy.random.randint(0, len(X)-1)
pattern = list(X[start])
print(''.join([int_to_char_mapping[value] for value in pattern]).replace(default_char,""))
result_sent = []
for i in range(500):
  x = numpy.reshape(pattern,(1,len(pattern),1))
  predict = lstm_model.predict(x,verbose=0)
  index = numpy.argmax(predict)
  result = int_to_char_mapping[index]
  result_sent.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

print("result sent :- ", "".join(result_sent).replace(default_char,""))

peared zara in a lovely blue andsilver dress, waiting for roderigo.  he came in gorgeous array, wit
result sent :-  hplumed cap, red cloak, chestnut lovelocks, a guitar, and the boots, ofcourse.  kneeling at the foot of the tower, he sang a serenade inmelting tones.  zara replied and, after a musical dialogue, consentedto fly.  then came the grand effect of the play.  roderigo produced arope ladder, with five steps to it, threw up one end, and invited zarato descend.  timidly she crept from her lattice, put her hand onroderigo's shoulder, and was about to leap gracefully down when alasalas for zara


In [77]:
# we can see that we are able to generate descent output from out swquence generator.
# The model we used which is Convolution comined with LSTM is faster to train and perform than using plain LSTM.
# Where only LSTM layers may take 700 seconds per epoc to train , it just took 50 seconds on an average.
# Next up is increase the vocabulary on word level and perform sequence/sentence generation.